In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain.document_loaders import PyPDFLoader
def load_pdf_files(files):
    # Assuming the provided 'files' list contains URLs
    pdf_url = files[0]

    # Read PDF content from the URL
    pdf_loader = PyPDFLoader(pdf_url)
    pages = pdf_loader.load_and_split()

    # If there are more files in the list, process them
    for i in range(1, len(files)):
        pdf_url = files[i]
        pdf_loader = PyPDFLoader(pdf_url)
        pages += pdf_loader.load_and_split()

    return pages

In [3]:
files=["./DataSourceFiles/IPC_186045.pdf"]
pages=load_pdf_files(files)
print("pages in PDF files: ",len(pages))

pages in PDF files:  187


In [41]:
from langchain.vectorstores import FAISS
from langchain.embeddings import  HuggingFaceInstructEmbeddings,HuggingFaceHubEmbeddings,HuggingFaceEmbeddings

# HFIembeddings = HuggingFaceInstructEmbeddings(model_name="WhereIsAI/UAE-Large-V1")    
HFIembeddings = HuggingFaceHubEmbeddings(repo_id="sentence-transformers/all-mpnet-base-v2")
# HFIembeddings = HuggingFaceHubEmbeddings(repo_id="sentence-transformers/paraphrase-MiniLM-L6-v2")
# HFIembeddings = HuggingFaceInstructEmbeddings(model_name="thenlper/gte-small",cache_folder="./Models/")
# HFIembeddings =HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
vectorstore = FAISS.from_documents(pages,embedding=HFIembeddings)
print("vectore store created")

You're using a different task than the one specified in the repository. Be sure to know what you're doing :)


vectore store created


In [48]:
# from langchain.vectorstores import Chroma
# from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

# embedding_function = SentenceTransformerEmbeddings(model_name="gte-small") #,cache_folder="./Models/")
# vectorstore = Chroma.from_documents(documents=pages,embedding=embedding_function,collection_name="law_collection")

In [49]:
from langchain.prompts import PromptTemplate
template = """
  You'r helpful AI assisant given the task to help people seeking law advise.
  You have to help person to use the Indian laws in legal manner.
  Answer in step by step in points by highlighting the sections of indian laws & constitution.
  Refuse to answer if it is not helping in legal affairs, also donot concoat anything.
{context}


Question: {question}  including section number and all related details.
Answer:"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [51]:

from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub
chain = RetrievalQA.from_chain_type(
    llm = HuggingFaceHub(repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1", model_kwargs={"temperature":0.8, "max_length":4096,"max_new_tokens":4096}),
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),    
    # return_source_documents=True,
    chain_type_kwargs={"prompt": prompt,"verbose": True},
)

In [59]:
# q="what is the punishment for robbery ?"
# q="who is judge ?"
q="one person has commited defamation, what should be punishment for it?"
# q="rioting has happened in neighbouring society. who will be responsible and what is the punishment for it"
# q="what is punishment for robbery and also murder ?"
# q="what are the rights of tenant and landlord ?"
# q="what is biscuit ?"
# q="person is selling adulterated drugs which is harmful to the health of people. person has also sold drug to the children below age of 15 what will be punishment for it?"
# q+=" with section number. add all details."
# q+=" including section number and all related details."
response = chain({"query":q, "early_stopping":True,"min_length":2000,"max_tokens":5000})
# print(response)
from write_in_file import generate_docx_with_bullets
from data_extracter import extract_data_from_response
# result,src_data,src_pg_nms=extract_data_from_response(response)
# generate_docx_with_bullets(heading=q,main_paragraph=response["result"],srcs=src_pg_nms,output_folder="./tmp/")
generate_docx_with_bullets(heading=q,main_paragraph=response["result"],output_folder="./tmp/")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

  You'r helpful AI assisant given the task to help people seeking law advise.
  You have to help person to use the Indian laws in legal manner.
  Answer in step by step in points by highlighting the sections of indian laws & constitution.
  Refuse to answer if it is not helping in legal affairs, also donot concoat anything.
111 
 504. Intentional insult with intent to provoke breach of the peace .—Whoever intentionally 
insults, and thereby gives provocation to any person, intending or knowing it to be likely that such 
provocation will cause him to break the public peace, or to commit any other offence, shall be punished 
with imprisonment of either description for a term which may extend to two yea rs, or with fine, or with 
both.  
1[505. Stateme nts conducing  to public mischief .—2[(1)] Whoever makes, publishes or  circulates 
any statement, rumour o r report, — 
(a) with int

In [ ]:
# def write_in_notepad(response):
#     from write_in_file import open_in_notepad
#     open_in_notepad(response["result"])


